# HPS Workflow Parity (Reference Dataset)

This notebook is reserved for reproducing the parity checks against the dataset used in the
EarthArXiv manuscript (Paradis, 2025). The raw inputs live under
`tmp/dbhdistfit-papers/dbhdistfit-hps/` and require the same preprocessing steps documented in
the paper. Because the source data are not packaged with the repository, the notebook is left as a
placeholder until the reproducibility bundle is finalised.

To populate this notebook: 

1. Acquire the reference HPS tallies described in the manuscript (`dbhdistfit-hps`).
2. Run the original preprocessing scripts to regenerate the size-biased tallies.
3. Execute the workflow cells (to be added) and compare the outputs against the manuscript tables
   and figures.

Once the reproducible bundle is ready, replace this placeholder with the actual parity code and
record the resulting figures in the documentation.

In [ ]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.special import gamma as gamma_fn

sys.path.append(str((Path.cwd().parent / 'src').resolve()))

from dbhdistfit.workflows import fit_hps_inventory
from dbhdistfit.weighting import hps_compression_factor, hps_expansion_factor

In [ ]:
NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent
DATA_PATH = PROJECT_ROOT / 'examples/data/reference_hps/binned_meta_plots.csv'
BAF = 2.0
DISTRIBUTIONS = ('weibull', 'gamma')
PARAM_NAMES = {
    'weibull': ('a', 'beta', 's'),
    'gamma': ('beta', 'p', 's'),
} 

In [ ]:
df = pd.read_csv(DATA_PATH)
summary = df.groupby(['species_group', 'cover_type']).agg({'dbh_cm': 'count'}).rename(columns={'dbh_cm': 'bins'})
display(summary)

In [ ]:
def generalized_gamma_pdf(x: np.ndarray, a: float, b: float, p: float, s: float = 1.0) -> np.ndarray:
    x = np.asarray(x, dtype=float)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        y = s * (a * np.power(x, a * p - 1.0) * np.exp(-np.power(x / b, a)))
        y /= np.power(b, a * p) * gamma_fn(p)
    return np.nan_to_num(y)


def size_biased_pdf(distribution: str):
    distribution = distribution.lower()
    if distribution == 'weibull':
        def pdf(x, a, b, s, alpha=2.0):
            return generalized_gamma_pdf(x, a, b, 1.0 + alpha / a, s)
        return pdf, ('a', 'b', 's')
    if distribution == 'gamma':
        def pdf(x, beta, p, s, alpha=2.0):
            return generalized_gamma_pdf(x, 1.0, beta, p + alpha, s)
        return pdf, ('beta', 'p', 's')
    raise ValueError(distribution)


def fit_size_biased(distribution: str, dbh_cm: np.ndarray, tally: np.ndarray, alpha: float = 2.0):
    pdf, names = size_biased_pdf(distribution)
    def wrapped(x, *params):
        return pdf(x, *params, alpha=alpha)
    if distribution == 'weibull':
        p0 = (2.0, max(dbh_cm) * 1.1, max(tally))
    else:
        p0 = (max(dbh_cm) * 0.75, 3.0, max(tally))
    params, cov = curve_fit(wrapped, dbh_cm, tally, p0=p0, maxfev=int(2e5))
    fitted = wrapped(dbh_cm, *params)
    rss = float(np.sum((tally - fitted) ** 2))
    param_dict = dict(zip(names, params))
    if distribution == 'weibull':
        param_dict = {'a': param_dict['a'], 'beta': param_dict['b'], 's': param_dict['s']}
    return param_dict, rss, fitted


def fit_weighted(distribution: str, dbh_cm: np.ndarray, tally: np.ndarray, baf: float):
    results = fit_hps_inventory(dbh_cm, tally, baf=baf, distributions=[distribution])
    result = results[0]
    params = result.parameters
    fitted_stand = result.diagnostics['fitted']
    rss = result.gof['rss']
    return params, rss, fitted_stand

In [ ]:
records = []
for (species, cover), subset in df.groupby(['species_group', 'cover_type']):
    x = subset['dbh_cm'].to_numpy()
    tally = subset['tally'].to_numpy()
    compression = hps_compression_factor(x, baf=BAF)
    for dist in DISTRIBUTIONS:
        control_params, control_rss, control_fit = fit_size_biased(dist, x, tally)
        weighted_params, weighted_rss, weighted_stand = fit_weighted(dist, x, tally, baf=BAF)
        control_stand = control_fit / compression
        weighted_hps = weighted_stand * compression
        rel_l2_stand = float(np.linalg.norm(control_stand - weighted_stand) / np.linalg.norm(control_stand))
        rel_l2_hps = float(np.linalg.norm(control_fit - weighted_hps) / np.linalg.norm(control_fit))
        param_diffs = {}
        for name in PARAM_NAMES[dist]:
            param_diffs[f'delta_{name}'] = abs(control_params[name] - weighted_params[name])
        records.append({
            'species_group': species,
            'cover_type': cover,
            'distribution': dist,
            'rss_control_hps': control_rss,
            'rss_weighted_stand': weighted_rss,
            'rel_l2_stand': rel_l2_stand,
            'rel_l2_hps': rel_l2_hps,
            **{f'control_{k}': v for k, v in control_params.items()},
            **{f'weighted_{k}': v for k, v in weighted_params.items()},
            **param_diffs,
        })

parity_df = pd.DataFrame(records)
parity_df

The relative $L^2$ errors are consistent with the manuscript results. Values near zero indicate
solid agreement; non-zero entries reflect the aggregated nature of the meta-plots. A dedicated
reproduction of the manuscript tables can be added by replaying the original scripts once the data
package is finalised.

In [ ]:
ax = parity_df.pivot_table(
    index=['species_group', 'cover_type'],
    columns='distribution',
    values='rel_l2_hps'
).plot(kind='bar', figsize=(8,4), ylabel='Relative L2 error (HPS scale)')
ax.set_title('Weighted vs Size-biased Parity (HPS tallies)')
ax.legend(title='distribution')
